## Taller de Procesamiento de Señales - TP3 Regresión Logística
### Alumno: Julián Stejman

--- 


a) 
1) Calcular la función inversa $\sigma^{-1}(p)$ con $p \in (0,1)$
2) Sea $p = \sigma(z)$ la función sigmoide, calcular la derivada $\sigma'(z)$ en términos de p.
3) Hallar una expresión analítica para la función costo y su gradiente. Tenga en cuenta el modelo asociado a una regresión logística de dos clases.

---

1) $$p = \sigma(z) = \frac{1}{1+e^{-z}}$$
$$
\frac{1}{p} = 1+e^{-z}
$$
$$
\frac{1}{p} -1= \frac{1-p}{p}=e^{-z}
$$
$$
log(\frac{1-p}{p}) = -z
$$
$$
\sigma^{-1}(p) = log(\frac{p}{1-p})
$$


2.  $$p'= \sigma'(z) = \left[ \frac{1}{1+e^{-z}}\right]' = \frac{e^{-z}}{(1+e^{-z})^2}$$
$$
= \frac{e^{-z}}{(1+e^{-z})^2} = \frac{1+ e^{-z} - 1}{(1+e^{-z})^2} = \frac{1+ e^{-z}}{(1+e^{-z})^2} - \frac{1}{(1+e^{-z})^2}
$$

$$
= \frac{1}{1+e^{-z}} - \frac{1}{(1+e^{-z})^2}
$$

$$
= \frac{1}{1+e^{-z}}\left[1- \frac{1}{1+e^{-z}} \right] = p(1-p)
$$

3. La función de costo que se utilizará es la Binary Cross-Entropy que tiene la siguiente forma:
$$
J(w,b) = -\frac{1}{N}\sum_{i=1}^N Y_i log(\sigma(\hat{Y_i})) + (1-Y_i)log(1-\sigma(\hat{Y_i}))
$$
$Y_i$ representa la etiqueta real de $X_i$ y la función $\sigma(\hat{Y_i})$ es análoga a la estimación de la probabilidad de $Y_i$ si uno considera que $\hat{Y}_i = w^TX_i + b$

Si se desea hallar $\nabla J(w,b)$ es equivalente a hacer:
$$
\nabla J(w,b) = -\frac{1}{n}\sum_{i=1}^N Y_i \nabla \left(log(\sigma(\hat{Y_i}))\right) + (1-Y_i)\nabla \left(log(1-\sigma(\hat{Y_i}))\right)
$$

Se busca entonces hallar esos gradientes:

 - $\nabla log(\sigma(\hat{Y_i})) = (1-\sigma(\hat{Y_i}))\left[X_i\;\; 1\right]^T$
-  $\nabla log(1-\sigma(\hat{Y_i})) = -\sigma(\hat{Y_i})\left[X_i\;\; 1\right]^T$

Entonces se tiene finalmente que el gradiente tiene esta forma:

$$
\nabla J(w,b) = \frac{1}{N}\sum_{i=0}^N \left(\sigma(\hat{Y_i}) - Y_i\right)\left[X_i\;\; 1\right]^T
$$